In [1]:
import numpy as np
import pandas as pd
from zipfile import ZipFile
import os
import rasterio
from scipy import stats

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import BoundaryNorm
from cartopy.feature import NaturalEarthFeature
import cartopy.crs as ccrs
import cartopy

In [2]:
lat = np.array([49.9166666666664 - i * 0.0416666666667 for i in range(357)])
lon = np.array([-105.0416666666507 + i * 0.0416666666667 for i in range(722)])
Lon, Lat = np.meshgrid(lon, lat)

am = rasterio.open('PRISM_tmin_stable_4kmD2_19810101_bil.bil')
a = am.read()[0, :357, 479:1201]
mask = np.where(a>-1000, 1, 0)

tmin81 = np.load('./prism/mw_tmin/1981_tmin.npz')['tmin']
t, y, x = tmin81.shape
y, x

(357, 722)

In [3]:
# var_mean = [np.zeros((y, x)) * np.nan for i in range(5)]
# var_std = [np.zeros((y, x)) * np.nan for i in range(5)]
# slope = [np.zeros((y, x)) * np.nan for i in range(5)]
# pvalue = [np.zeros((y, x)) * np.nan for i in range(5)]
# X = np.linspace(1, 38, 38)
# name = ['year','sidegreen','bloom','pdays','yield']

# for i in range(y):
#     for j in range(x):
#         if mask[i, j]:
#             df = pd.read_table(f'./GDD/output_Cherry/yearly/{Lat[i, j]:.2f}_{Lon[i, j]:.2f}_yearly.txt', delim_whitespace=True, 
#                                    names=('year','sidegreen','bloom','pdays','yield'))
#             for ii in range(1,5):
#                 var_mean[ii-1][i,j] = df[name[ii]].mean()
                
#                 var_detrend = df[name[ii]] - var_mean[ii-1][i, j]
#                 var_std[ii-1][i,j] = np.sqrt(((var_detrend - var_detrend.mean())**2).sum() / 38.)
                
#                 r = stats.linregress(X, df[name[ii]])
#                 slope[ii-1][i, j] = r.slope
#                 pvalue[ii-1][i, j] = r.pvalue
            
#             spring = df['bloom'] - df['sidegreen']
#             var_mean[-1][i,j] = spring.mean()
#             var_detrend = spring - var_mean[-1][i, j]
#             var_std[-1][i,j] = np.sqrt(((var_detrend - var_detrend.mean())**2).sum() / 38.)    
#             r = stats.linregress(X, spring)
#             slope[-1][i, j] = r.slope
#             pvalue[-1][i, j] = r.pvalue

# np.save('./GDD/var_yearly_Cherry', np.array(var_mean))
# np.save('./GDD/var_std_Cherry', np.array(var_std))
# np.save('./GDD/var_yearly__slope_Cherry', np.array(slope))
# np.save('./GDD/var_yearly_pvalue_Cherry', np.array(pvalue))

In [3]:
var_mean = np.load('./GDD/var_yearly_Cherry.npy')
var_std = np.load('./GDD/var_std_Cherry.npy')
var_slope = np.load('./GDD/var_yearly__slope_Cherry.npy')
var_pvalue = np.load('./GDD/var_yearly_pvalue_Cherry.npy')

In [4]:
data = []
pvalue = []
data.append(var_mean[0, :, :])
data.append(var_std[0, :, :])
data.append(var_slope[0, :, :])
pvalue.append(var_pvalue[0, :, :])
data.append(var_mean[1, :, :])
data.append(var_std[1, :, :])
data.append(var_slope[1, :, :])
pvalue.append(var_pvalue[1, :, :])
data.append(var_mean[4, :, :])
data.append(var_std[4, :, :])
data.append(var_slope[4, :, :])
pvalue.append(var_pvalue[4, :, :])

In [45]:
extent = [-105, -75, 34, 49]
fig, axs = plt.subplots(3, 3, figsize=(15, 9.5))
ll = [0, 0, -.6, 0, 0, -.6, 0, 0, -.6]
lr = [160, 16, .6, 160, 16, .6, 40, 16, .6]
step = [40, 2, .2, 40, 2, .2, 10, 2, .2]
title = ['(a) Side Green mean', '(b) Side Green std', '(c) Side Green slope', '(d) Full Bloom mean', 
         '(e) Full Bloom std', '(f) Full Bloom slope', '(g) Spring Duration mean', '(h) Spring Duration std', 
         '(i) Spring Duration slope']
pc = []
for i in range(9):
    ax = plt.subplot(3,3,i+1, projection=ccrs.AlbersEqualArea(np.mean(extent[:2]), np.mean(extent[2:])))
    ax.set_extent(extent)
    ax.add_feature(cartopy.feature.BORDERS, lw=.1, linestyle=':')
    ax.add_feature(cartopy.feature.COASTLINE, lw=.1, linestyle=':')
    ax.add_feature(cartopy.feature.STATES.with_scale('10m'), lw=.5)
    ax.add_feature(cartopy.feature.LAKES, edgecolor='black', facecolor='white', lw=.3)
    
    levels = MaxNLocator(nbins=100).tick_values(ll[i], lr[i])
    if i%3-2:
        cmap = plt.get_cmap('viridis')
    else:
        cmap = plt.get_cmap('bwr')
    norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

    pc.append(plt.pcolormesh(Lon, Lat, data[i], cmap=cmap, norm=norm, transform=ccrs.PlateCarree()))
    if i%3 == 2:
        plt.scatter(Lon, Lat, np.where(pvalue[int((i-2)/3)]<0.05, 1, np.nan), 'grey', alpha=.1, transform=ccrs.PlateCarree())
    ax.text(.99,.95,title[i], fontsize=13, fontweight='bold',horizontalalignment='right',transform=ax.transAxes)
plt.subplots_adjust(bottom=0.02, top=.98, left=0.02, right=.98,
                    wspace=0.005, hspace=0.005)
i = 0
cb_ax = fig.add_axes([.005, 0.345, 0.01, 0.64])
cbar = fig.colorbar(pc[i], cax=cb_ax, ticks=np.arange(ll[i], lr[i]+step[i], step[i]), extend='max')
cb_ax.yaxis.set_ticks_position('left')
cb_ax.tick_params(labelsize=12) 
i = 2
cb_ax = fig.add_axes([0.985, .02, 0.01, 0.96])
cbar = fig.colorbar(pc[i], cax=cb_ax, ticks=np.arange(ll[i], lr[i]+step[i], step[i]), extend='both')
cb_ax.tick_params(labelsize=12)
i = 6
cb_ax = fig.add_axes([0.005, .02, 0.01, 0.32])
cbar = fig.colorbar(pc[i], cax=cb_ax, ticks=np.arange(ll[i], lr[i]+step[i], step[i]), extend='max')
cb_ax.yaxis.set_ticks_position('left')
cb_ax.tick_params(labelsize=12)
i = 7
cb_ax = fig.add_axes([0.34, .005, 0.32, 0.01])
cbar = fig.colorbar(pc[i], cax=cb_ax, ticks=np.arange(ll[i], lr[i]+step[i], step[i]), extend='max', orientation="horizontal")
cb_ax.tick_params(labelsize=12)

plt.savefig('./plot/AgPaper/fig1.png', bbox_inches='tight')
plt.close()

<ipython-input-45-207253e8bbbc>:27: RuntimeWarning: invalid value encountered in less
  plt.scatter(Lon, Lat, np.where(pvalue[int((i-2)/3)]<0.05, 1, np.nan), 'grey', alpha=.1, transform=ccrs.PlateCarree())


In [46]:
data = []
pvalue = []
data.append(var_mean[2, :, :])
data.append(var_std[2, :, :])
data.append(var_slope[2, :, :])
pvalue.append(var_pvalue[2, :, :])
data.append(var_mean[3, :, :])
data.append(var_std[3, :, :])
data.append(var_slope[3, :, :])
pvalue.append(var_pvalue[3, :, :])

In [ ]:
# extent = [-105, -75, 34, 49]
# fig, axs = plt.subplots(3, 2, figsize=(13, 8))
# ll = [0, 0, -.6, 0, 0, -.6, 0, 0, -.6]
# lr = [160, 16, .6, 160, 16, .6, 40, 16, .6]
# step = [40, 2, .2, 40, 2, .2, 10, 2, .2]
# title = ['(a) Side Green mean', '(b) Side Green std', '(c) Side Green slope', '(d) Full Bloom mean', 
#          '(e) Full Bloom std', '(f) Full Bloom slope', '(g) Spring Duration mean', '(h) Spring Duration std', 
#          '(i) Spring Duration slope']
# pc = []
# for i in range(9):
#     ax = plt.subplot(3,3,i+1, projection=ccrs.AlbersEqualArea(np.mean(extent[:2]), np.mean(extent[2:])))
#     ax.set_extent(extent)
#     ax.add_feature(cartopy.feature.BORDERS, lw=.1, linestyle=':')
#     ax.add_feature(cartopy.feature.COASTLINE, lw=.1, linestyle=':')
#     ax.add_feature(cartopy.feature.STATES.with_scale('10m'), lw=.5)
#     ax.add_feature(cartopy.feature.LAKES, edgecolor='black', facecolor='white', lw=.3)
    
#     levels = MaxNLocator(nbins=100).tick_values(ll[i], lr[i])
#     if i%3-2:
#         cmap = plt.get_cmap('viridis')
#     else:
#         cmap = plt.get_cmap('bwr')
#     norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

#     pc.append(plt.pcolormesh(Lon, Lat, data[i], cmap=cmap, norm=norm, transform=ccrs.PlateCarree()))
#     if i%3 == 2:
#         plt.scatter(Lon, Lat, np.where(pvalue[int((i-2)/3)]<0.05, 1, np.nan), 'grey', alpha=.1, transform=ccrs.PlateCarree())
#     ax.text(.99,.95,title[i], fontsize=13, fontweight='bold',horizontalalignment='right',transform=ax.transAxes)
# plt.subplots_adjust(bottom=0.02, top=.98, left=0.02, right=.98,
#                     wspace=0.005, hspace=0.005)
# i = 0
# cb_ax = fig.add_axes([.005, 0.345, 0.01, 0.64])
# cbar = fig.colorbar(pc[i], cax=cb_ax, ticks=np.arange(ll[i], lr[i]+step[i], step[i]), extend='max')
# cb_ax.yaxis.set_ticks_position('left')
# cb_ax.tick_params(labelsize=12) 
# i = 2
# cb_ax = fig.add_axes([0.985, .02, 0.01, 0.96])
# cbar = fig.colorbar(pc[i], cax=cb_ax, ticks=np.arange(ll[i], lr[i]+step[i], step[i]), extend='both')
# cb_ax.tick_params(labelsize=12)
# i = 6
# cb_ax = fig.add_axes([0.005, .02, 0.01, 0.32])
# cbar = fig.colorbar(pc[i], cax=cb_ax, ticks=np.arange(ll[i], lr[i]+step[i], step[i]), extend='max')
# cb_ax.yaxis.set_ticks_position('left')
# cb_ax.tick_params(labelsize=12)
# i = 7
# cb_ax = fig.add_axes([0.34, .005, 0.32, 0.01])
# cbar = fig.colorbar(pc[i], cax=cb_ax, ticks=np.arange(ll[i], lr[i]+step[i], step[i]), extend='max', orientation="horizontal")
# cb_ax.tick_params(labelsize=12)

# # plt.savefig('./plot/AgPaper/fig2.png', bbox_inches='tight')
# # plt.close()